# Continuous-time Markov processes

In [ ]:
import logging
import random

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

logging.getLogger().setLevel(logging.WARN)

In [ ]:
def evolution(fun, ts, state0, resample=None):
    """Event-driven simulation of a continuous-time Markov process."""
    # TODO specify max rate/min step, to interrupt in case of very high rates
    if type(ts) in (int, float):
        ts = np.linspace(0, ts)
    
    state = state0
    t = ts[0]
    history = [[0, state0]]
    
    while t < ts[-1]:
        transitions = fun(t, state)
        transitions = {k: transitions[k] for k in transitions if transitions[k] > 0}
        logging.info(transitions)
        if not transitions:
            break
        
        times_to_move = {k: np.random.exponential(1/transitions[k]) for k in transitions}
        logging.info(times_to_move)       

        new_state = min(times_to_move, key=lambda k: times_to_move[k])
        time_elapsed = times_to_move[new_state]
        t += time_elapsed
        state = new_state
        history.append([t, state])
                
    if resample is None:
        return np.array(history)
    
    resampled_history = []
    for t in resample:
        last_event = max([h for h in history if h[0] <= t], key=lambda h: h[0])
        resampled_history.append([t, last_event[1]])
        
    return np.array(resampled_history)

## Stochastic Lotka-Volterra

In [ ]:
def transition_fun(t, n):
    x, y = n
    s = .001
    return {
        (x + s, y): 2/3 * x,
        (x - 4/3*s, y+s): x * y,
        (x, y - s): y
    }

history = evolution(transition_fun, 20000, [1, 0.75],
                    resample=np.linspace(0, 20000, 101))

fig = plt.figure(figsize=(15, 6))
plt.plot([h[0] for h in history],
         [h[1] for h in history])
plt.grid()
plt.gca().set_ylim([0, None])
plt.show()

In [ ]:
initial_states = [(1 + r*np.cos(th), 1 + r*np.sin(th)) 
                  for r in np.linspace(.1, .4, 4)
                  for th in np.linspace(0, 2*np.pi, 9)[:-1]]
histories = [evolution(transition_fun, 10000, s, np.linspace(0, 10000, 201)) for s in initial_states]

In [ ]:
average_x = [np.average([h[t][1][0] for h in histories]) for t in range(len(histories[0]))]
average_y = [np.average([h[t][1][1] for h in histories]) for t in range(len(histories[0]))]

In [ ]:
@interact(step=widgets.IntSlider(min=0, max=len(histories[0])-1, value=0, continuous_update=False))
def plot_evolution(step):
    final_states = [h[step][1] for h in histories]

    plt.figure(figsize=[8,8])
    plt.plot(
        average_x, average_y, 'r-',
        [x[0] for x in final_states], 
        [x[1] for x in final_states],
        'k.',
        average_x[step], average_y[step], 'ko'
    )
    plt.gca().set_xlim([0, 2])
    plt.gca().set_ylim([0, 2])
    plt.gca().set_aspect('equal')
    plt.gca().grid()
    # = evolution(transition_fun, 60, [50, 0])

## Stochastic harmonic oscillator

In [ ]:
def transition_fun(t, s):
    x, y = s
    return {
        (x + 1, y): y,
        (x - 1, y): -y,
        (x, y + 1): -x,
        (x, y - 1): x
    }

history = evolution(transition_fun, 40, [40, 40],
                    resample=np.linspace(0, 40, 121))

plt.plot([h[0] for h in history],
         [h[1] for h in history])
plt.show()


plt.plot([h[1][0] for h in history],
        [h[1][1] for h in history])
plt.show()

In [ ]:
initial_states = [(30 + r*np.cos(th), 40 + r*np.sin(th)) 
                  for r in np.linspace(1, 5, 5)
                  for th in np.linspace(0, 2*np.pi, 9)]

histories = [evolution(transition_fun, 25, s, np.linspace(0, 25, 161)) for s in initial_states]
average_x = [np.average([h[t][1][0] for h in histories]) for t in range(len(histories[0]))]
average_y = [np.average([h[t][1][1] for h in histories]) for t in range(len(histories[0]))]

In [ ]:
@interact(step=widgets.IntSlider(min=0, max=len(histories[0])-1, value=0, continuous_update=False))
def plot_evolution(step):
    final_states = [h[step][1] for h in histories]

    plt.figure(figsize=[8,8])
    plt.plot(
        average_x, average_y, 'r-',
        [x[0] for x in final_states], 
             [x[1] for x in final_states],
            'k.',
        average_x[step], average_y[step], 'k+',)
    plt.gca().set_xlim([-60, 60])
    plt.gca().set_ylim([-60, 60])
    plt.gca().set_aspect('equal')
    plt.gca().grid()
    

## Decay with regeneration

In [ ]:
def transition_fun(t, s):
    x, y = s
    return {
        (x + 1, y): 1 - 0.05*x + y,
        (x - 1, y): 1 + 0.05*x - y,
        (x, y + 1): 1 - 0.05*y - x,
        (x, y - 1): 1 + 0.05*y + x
    }

history = evolution(transition_fun, 100, [10, 10],
                    resample=np.linspace(0, 100, 151))

plt.figure(figsize=(14, 5))
plt.plot([h[0] for h in history],
         [h[1] for h in history],
        )
plt.show()


plt.plot([h[1][0] for h in history],
        [h[1][1] for h in history])
plt.show()

In [ ]:
initial_states = [(60 + r*np.cos(th), 60 + r*np.sin(th)) 
                  for r in np.linspace(1, 5, 5)
                  for th in np.linspace(0, 2*np.pi, 9)[:-1]]

histories = [evolution(transition_fun, 100, s, np.linspace(0, 100, 251)) for s in initial_states]
average_x = [np.average([h[t][1][0] for h in histories]) for t in range(len(histories[0]))]
average_y = [np.average([h[t][1][1] for h in histories]) for t in range(len(histories[0]))]

In [ ]:
@interact(step=widgets.IntSlider(min=0, max=len(histories[0])-1, value=0, continuous_update=False))
def plot_evolution(step):
    final_states = [h[step][1] for h in histories]

    plt.figure(figsize=[8,8])
    plt.plot(
        average_x, average_y, 'r-',
        [x[0] for x in final_states], 
        [x[1] for x in final_states],
        'k.',
        average_x[step], average_y[step], 'k+'
    )
    plt.gca().set_xlim([-120, 120])
    plt.gca().set_ylim([-120, 120])
    plt.gca().set_aspect('equal')
    plt.gca().grid()
    